In [1]:
!pip install transformers datasets torch scikit-learn


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.1/542.1 kB 3.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_6

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from imblearn.over_sampling import SMOTE
from sklearn.pipeline import Pipeline
from transformers import BertTokenizer, BertForSequenceClassification
from tqdm import tqdm
import torch
from torch.utils.data import DataLoader, TensorDataset
from torch.nn import functional as F
import matplotlib.pyplot as plt

In [2]:
import torch
import pandas as pd
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset


# Check if GPU is available
device = torch.device("cpu")

In [3]:
model_name = 'bert-base-multilingual-cased'
num_classes = 4
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)
tokenizer = BertTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

In [4]:
df = pd.read_csv('/content/balanced_data_with_value_counts.csv')
df.columns

Index(['Unnamed: 0', 'review_content', 'student1_email', 'student1annotation',
       'student2_email', 'student2annotation', 'student3_email',
       'student3annotation', 'annotation_count', 'agreement_type',
       'final_label', 'processed_review_content'],
      dtype='object')

In [11]:
df['final_label'].value_counts()

final_label
4    2846
1    2835
3    2604
2    2428
Name: count, dtype: int64

In [5]:
from sklearn.model_selection import train_test_split

# Assuming you want an 80-10-10 split for train, validation, and test sets
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

# Separate features and labels for each split
X_train, y_train = train_df['processed_review_content'], train_df['final_label']
X_val, y_val = val_df['processed_review_content'], val_df['final_label']
X_test, y_test = test_df['processed_review_content'], test_df['final_label']

In [6]:
# Convert to strings and handle missing values
X_train = X_train.astype(str).fillna('')
X_val = X_val.astype(str).fillna('')
X_test = X_test.astype(str).fillna('')

def preprocess_data(texts, labels, max_length):
    encodings = tokenizer(texts.tolist(), truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
    labels = torch.tensor(labels.tolist()).to(device)
    dataset = TensorDataset(encodings.input_ids.to(device), encodings.attention_mask.to(device), labels)
    return dataset

max_length = 256  # Set your desired max sequence length

train_dataset = preprocess_data(X_train, y_train, max_length)
val_dataset = preprocess_data(X_val, y_val, max_length)
test_dataset = preprocess_data(X_test, y_test, max_length)


In [8]:
batch_size = 8

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)


In [9]:
# Define optimizer and loss function
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()


In [5]:
import torch
from transformers import BertForSequenceClassification, BertTokenizer
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
import pandas as pd

# Load your dataset
# Assuming you have a DataFrame named df with 'processed_review_content' and 'final_label' columns
# Example:
# df = pd.read_csv('your_dataset.csv')

# Adjust label encoding (subtract 1 from each label)
df['final_label'] = df['final_label'] - 1

# Verify updated labels
print(df['final_label'].value_counts())

# BERT model and tokenizer initialization
model_name = 'bert-base-multilingual-cased'
num_classes = 4
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)
tokenizer = BertTokenizer.from_pretrained(model_name)

# Data preprocessing
train_df, temp_df = train_test_split(df, test_size=0.2, random_state=42)
val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)

X_train, y_train = train_df['processed_review_content'], train_df['final_label']
X_val, y_val = val_df['processed_review_content'], val_df['final_label']
X_test, y_test = test_df['processed_review_content'], test_df['final_label']

X_train = X_train.astype(str).fillna('')
X_val = X_val.astype(str).fillna('')
X_test = X_test.astype(str).fillna('')

def preprocess_data(texts, labels, max_length):
    encodings = tokenizer(texts.tolist(), truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')
    labels = torch.tensor(labels.tolist())
    dataset = TensorDataset(encodings.input_ids, encodings.attention_mask, labels)
    return dataset

max_length = 256
train_dataset = preprocess_data(X_train, y_train, max_length)
val_dataset = preprocess_data(X_val, y_val, max_length)
test_dataset = preprocess_data(X_test, y_test, max_length)

batch_size = 8

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# Model training
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

optimizer = torch.optim.AdamW(model.parameters(), lr=2e-5)

num_epochs = 20
early_stopping_patience = 3
best_val_loss = float('inf')
epochs_since_last_improvement = 0

for epoch in range(num_epochs):
    model.train()
    for batch in train_loader:
        optimizer.zero_grad()
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

    model.eval()
    val_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for batch in val_loader:
            input_ids, attention_mask, labels = batch
            input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            val_loss += loss.item()
            predicted_labels = torch.argmax(outputs.logits, dim=1)
            correct += (predicted_labels == labels).sum().item()
            total += labels.size(0)

    val_accuracy = correct / total
    avg_val_loss = val_loss / len(val_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"Validation Accuracy: {val_accuracy:.4f}")
    print(f"Avg. Validation Loss: {avg_val_loss:.4f}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        epochs_since_last_improvement = 0
    else:
        epochs_since_last_improvement += 1
        if epochs_since_last_improvement >= early_stopping_patience:
            print("Early stopping triggered. Stopping training.")
            break

# Save the model if needed
# torch.save(model.state_dict(), 'bert_model.pt')


final_label
3    2846
0    2835
2    2604
1    2428
Name: count, dtype: int64


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/625 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

Epoch [1/20]
Validation Accuracy: 0.4781
Avg. Validation Loss: 0.9561
Epoch [2/20]
Validation Accuracy: 0.4939
Avg. Validation Loss: 0.9410
Epoch [3/20]
Validation Accuracy: 0.5275
Avg. Validation Loss: 0.9311
Epoch [4/20]
Validation Accuracy: 0.5406
Avg. Validation Loss: 0.9406
Epoch [5/20]
Validation Accuracy: 0.5686
Avg. Validation Loss: 0.9278
Epoch [6/20]
Validation Accuracy: 0.6377
Avg. Validation Loss: 0.7572
Epoch [7/20]
Validation Accuracy: 0.6667
Avg. Validation Loss: 0.6890
Epoch [8/20]
Validation Accuracy: 0.6797
Avg. Validation Loss: 0.6614
Epoch [9/20]
Validation Accuracy: 0.6732
Avg. Validation Loss: 0.7014
Epoch [10/20]
Validation Accuracy: 0.6928
Avg. Validation Loss: 0.8715
Epoch [11/20]
Validation Accuracy: 0.6835
Avg. Validation Loss: 0.9338
Early stopping triggered. Stopping training.


In [6]:
from google.colab import drive
drive.mount('/content/drive')

model_path = "/content/drive/MyDrive/mbert_model.pth"
torch.save(model.state_dict(), model_path)

Mounted at /content/drive


In [7]:
from transformers import BertForSequenceClassification

# Load the saved model
model2 = BertForSequenceClassification.from_pretrained(model_name, num_labels=num_classes)
model2.load_state_dict(torch.load("/content/drive/MyDrive/mbert_model.pth"))
model2.to(device)

# Evaluate the model
model2.eval()
test_preds = []
test_true_labels = []

with torch.no_grad():
    for batch in test_loader:
        input_ids, attention_mask, labels = batch
        input_ids, attention_mask, labels = input_ids.to(device), attention_mask.to(device), labels.to(device)
        outputs = model2(input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        predicted_labels = torch.argmax(logits, dim=1)
        test_preds.extend(predicted_labels.cpu().numpy())
        test_true_labels.extend(labels.cpu().numpy())

test_accuracy = accuracy_score(test_true_labels, test_preds)
print(f"Test Accuracy: {test_accuracy:.4f}")

# Generate classification report
class_names = ["Class 0", "Class 1", "Class 2", "Class 3"]  # Replace with your actual class names
report = classification_report(test_true_labels, test_preds, target_names=class_names, digits=4)
print("Classification Report:")
print(report)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Test Accuracy: 0.6959
Classification Report:
              precision    recall  f1-score   support

     Class 0     0.9224    0.8045    0.8594       266
     Class 1     0.5385    0.5155    0.5267       258
     Class 2     0.7659    0.9672    0.8548       274
     Class 3     0.5425    0.4891    0.5144       274

    accuracy                         0.6959      1072
   macro avg     0.6923    0.6941    0.6889      1072
weighted avg     0.6929    0.6959    0.6900      1072



config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cpu and cuda:0! (when checking argument for argument index in method wrapper_CUDA__index_select)

In [ ]:
!pip install transformers[torch] accelerate -U


In [ ]:
torch.cuda.is_available()

True

GPU is available


In [ ]:
!nvcc --version


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2023 NVIDIA Corporation
Built on Tue_Aug_15_22:02:13_PDT_2023
Cuda compilation tools, release 12.2, V12.2.140
Build cuda_12.2.r12.2/compiler.33191640_0


In [ ]:
!nvidia-smi


Sat Jun  8 10:35:34 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   48C    P0              27W /  70W |   5439MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--